In [1]:
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras import models
import imgaug.augmenters as iaa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

2023-07-27 21:42:13.802637: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 21:42:13.836752: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 21:42:13.837384: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 21:42:14.531453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_train = x_train.astype('float32')/255.0
x_test = x_test.reshape(-1, 28, 28, 1)

x_test = x_test.astype('float32')/255.0


In [3]:
x_train1, x_val, y_train1, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [4]:
y_train1_categorical = tf.keras.utils.to_categorical(y_train1, num_classes=10)
y_val_categorical = tf.keras.utils.to_categorical(y_val, num_classes=10)
y_test_categorical = tf.keras.utils.to_categorical(y_test, num_classes=10)


In [5]:


model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape = (28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(10, activation= 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
augmenter = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 1.0))
])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
   preprocessing_function=augmenter.augment_image
)

datagen.fit(x_train1)


In [8]:
#aug_pipeline = iaa.Sequential([
 #   iaa.SomeOf((0, 3), [
   #     iaa.Fliplr(1.0),  # horizontally flip
  #      iaa.Flipud(1.0),  # Vertical flip
    #    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),  # sharpen images
     #   iaa.Crop(percent=(0, 0.4)),
      #  iaa.Sometimes(0.5, iaa.Affine(rotate=5)),
       # iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
        #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),
    #])
#],
#random_order=True  
#)
#x_train_augmented = []
#for image in x_train:
 #   augmented = aug_pipeline(image=image)*/
  #  x_train_augmented.append(augmented['image'])

In [9]:
model.fit(datagen.flow(x_train1, y_train1_categorical,  batch_size=32),epochs=5,validation_data=(x_val, y_val_categorical))


Epoch 1/5
1688/1688 [==============================] - 30s 18ms/step - loss: 0.5094 - accuracy: 0.8303 - val_loss: 0.1512 - val_accuracy: 0.9540
Epoch 2/5
1688/1688 [==============================] - 29s 17ms/step - loss: 0.1969 - accuracy: 0.9391 - val_loss: 0.1211 - val_accuracy: 0.9645
Epoch 3/5
1688/1688 [==============================] - 29s 17ms/step - loss: 0.1478 - accuracy: 0.9543 - val_loss: 0.0695 - val_accuracy: 0.9795
Epoch 4/5
1688/1688 [==============================] - 29s 17ms/step - loss: 0.1304 - accuracy: 0.9595 - val_loss: 0.0562 - val_accuracy: 0.9828
Epoch 5/5
1688/1688 [==============================] - 29s 17ms/step - loss: 0.1121 - accuracy: 0.9659 - val_loss: 0.1059 - val_accuracy: 0.9648


In [10]:
#test_loss, test_acc = model.evaluate(x_test, y_test_categorical)
#print(f'Test accuracy: {test_acc}')



In [11]:
val_scores = model.evaluate(x_val, y_val_categorical, verbose=0)
val_accuracy, val_loss = val_scores[1], val_scores[0]

test_scores = model.evaluate(x_test, y_test_categorical, verbose=0)
test_accuracy, test_loss = test_scores[1], test_scores[0]

y_val_pred = model.predict(x_val)
y_val_pred_labels = tf.argmax(y_val_pred, axis=1)
val_precision = precision_score(tf.argmax(y_val_categorical, axis=1), y_val_pred_labels, average='weighted')
val_recall = recall_score(tf.argmax(y_val_categorical, axis=1), y_val_pred_labels, average='weighted')
val_f1 = f1_score(tf.argmax(y_val_categorical, axis=1), y_val_pred_labels, average='weighted')

y_test_pred = model.predict(x_test)
y_test_pred_labels = tf.argmax(y_test_pred, axis=1)
test_precision = precision_score(tf.argmax(y_test_categorical, axis=1), y_test_pred_labels, average='weighted')
test_recall = recall_score(tf.argmax(y_test_categorical, axis=1), y_test_pred_labels, average='weighted')
test_f1 = f1_score(tf.argmax(y_test_categorical, axis=1), y_test_pred_labels, average='weighted')


313/313 [==============================] - 1s 2ms/step


In [12]:
print(f'Validation Accuracy: {val_accuracy:.4f}, Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}, Loss: {test_loss:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}')

Validation Accuracy: 0.9648, Loss: 0.1059, Precision: 0.9673, Recall: 0.9648, F1 Score: 0.9652
Test Accuracy: 0.9616, Loss: 0.1112, Precision: 0.9659, Recall: 0.9616, F1 Score: 0.9623
